# Homework 1
**Advanced Optimization Models** \
Lorena Tassone, 18-700-237

Python implementation of ULS models (MIP models and LP relaxations) using Gurobi optimizer.

In [1]:
# import libraries
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import random

## Models

### Uncapacitated Lot-Sizing (ULS)

Constraints and objective:
$$
\begin{aligned}
\min \sum_{t=1}^n p_t y_t+\sum_{t=1}^n h_t s_t & +\sum_{t=1}^n f_t x_t \\
s_{t-1}+y_t & =d_t+s_t \text { for } t=1, \ldots, n \\
y_t & \leq M x_t \text { for } t=1, \ldots, n \\
s_0=0, s_t, y_t & \geq 0, x_t \in\{0,1\} \text { for } t=1, \ldots, n
\end{aligned}
$$

$f_t$ is the fixed cost of producing in period t. \
$p_t$ is the unit production cost in period t. \
$h_t$ is the unit storage cost in period t. \
$d_t$ is the demand in period t. 

$y_t$ is the amount produced in period t. \
$s_t$ is the stock at the end of period t. \
$x_t$ = 1 if production occurs in t, and $x_t$ = 0 otherwise. 

To handle fix costs the Big-M notation is used by choosing a large value M as upper bound. M could also be calculated based on the problem data, e.g. $M=\sum_{t=1}^n d_t$.

In [2]:
# ULS model 1
lp_relax = 0 # if (lp_relax == 0), then we solve the original problem; if (lp_relax == 1), then we solve its lp relaxation

def solve_ULS_1(f, p, h, d, n, M, lp_relax):
    
    # Create new model
    ULS_model = gp.Model('ULS 1') 
    ULS_model.setParam('OutputFlag', 0) # Suppress solver output
    
    # Define variables
    x = [] # x_t = 1 if production occurs in t, and x_t = 0 otherwise.
    if (lp_relax == 0):
        x = ULS_model.addVars(n, vtype=GRB.BINARY, name="x")
    if (lp_relax == 1):
        x = ULS_model.addVars(n, vtype=GRB.CONTINUOUS, name="x")
    
    y = ULS_model.addVars(n, vtype=GRB.CONTINUOUS, name="y") # the amount produced in period t
    s = ULS_model.addVars(n + 1, vtype=GRB.CONTINUOUS, name="s") # stock at the end of period t - 1!
    
    # Set objective function                      
    obj_fn = gp.quicksum(f[t] * x[t] for t in range(n)) + \
             gp.quicksum(p[t] * y[t] for t in range(n)) + \
             gp.quicksum(h[t] * s[t+1] for t in range(n))                                                                                                            
    ULS_model.setObjective(obj_fn, GRB.MINIMIZE)

    # Add constraints 
    for t in range(n):
        ULS_model.addConstr(s[t] + y[t] == d[t] + s[t + 1], name="cons_flow") # s[t]=s_{t-1}
        ULS_model.addConstr(y[t] <= M * x[t], name="cons_prod")
        ULS_model.addConstr(y[t] >= 0, name="cons_prod_min")
        ULS_model.addConstr(s[t + 1] >= 0, name="cons_stock_min")

    ULS_model.addConstr(s[0] == 0, name="cons_init_stock")

    # LP relaxation case
    if (lp_relax == 1):
        # ULS_model.relax()          # Creates relaxation of MIP model by transforming integer var into continuous var
        ULS_model.Params.Method = 1  # Use dual simplex for LP relaxation
#        for t in range(n):          # manual relaxation
#            ULS_model.addConstr(x[t] >= 0, name="x_lb")
#            ULS_model.addConstr(x[t] <= 1, name="x_ub")

    # Solve model
    ULS_model.optimize()

    # Output results
    obj_opt = ULS_model.objVal
    
    print("\nOutput")
    print(60*'=')
    
    if ULS_model.status == GRB.OPTIMAL:
        print("Optimal objective value:", obj_opt)
        print("Production decisions (x):", [x[t].x for t in range(n)])
        print("Produced quantity (y):", [y[t].x for t in range(n)])
        print("End-of-period inventory (s):", [s[t].x for t in range(n + 1)])
    else:
        print("No optimal solution found")
    print("\n")  
    
    return obj_opt

### Extended Uncapacitated Lot-Sizing (ULS)

Constraints and objective:
$$
\min \sum_{i=1}^n \sum_{t=i}^n c_i w_{i t}+\sum_{t=1}^n f_t x_t - K \\
\begin{aligned}
s_{t-1}+y_t & =d_t+s_t \quad \text { for } t=1, \ldots, n \\
\sum_{i=1} w_{i t} & =d_t \quad \text { for } t=1, \ldots, n \\
w_{i t} & \leq d_t x_i \quad \text { for all } i, t, i \leq t \\
y_i & =\sum_{t=i}^n w_{i t} \text { for } i=1, \ldots, n \\
w_{i t} & \geq 0 \text { for all } i, t, i \leq t \\
0 & \leq x_t \leq 1 \text { for all } t 
\end{aligned}
$$

The basic ULS model is extended by adding the variable $w_{it}$, which is the amount produced in period i to satisfy demand in period t.

Modified costs $c_i$ and constant $K$ are used. \
Since we impose that $s_n=0$, as done in the textbook, we can tighten the variable upper bound constraints to $y_t \leq\left(\sum_{i=t}^n d_i\right) x_t$. By substituting $s_t=\sum_{i=1}^t y_i-\sum_{i=1}^t d_i$ the objective function can be rewritten as $\sum_{t=1}^n c_t y_t+\sum_{t=1}^n f_t x_t-K$ where $c_t=p_t+h_t+\cdots+h_n$ and the constant $K=\sum_{t=1}^n h_t\left(\sum_{i=1}^t d_i\right)$.

In [3]:
# ULS model 2: extended
lp_relax = 0 # if (lp_relax == 0), then we solve the original problem; if (lp_relax == 1), then we solve its lp relaxation

def solve_ULS_2(f, p, h, d, n, lp_relax):
    
    # Create new model
    ULS_model = gp.Model('ULS 2') 
    ULS_model.setParam('OutputFlag', 0) # Suppress solver output

    # Define variables
    x = [] # x_t = 1 if production occurs in t, and x_t = 0 otherwise.
    if (lp_relax == 0):
        x = ULS_model.addVars(n, vtype=GRB.BINARY, name="x")
    if (lp_relax == 1):
        x = ULS_model.addVars(n, vtype=GRB.CONTINUOUS, name="x")

    w = ULS_model.addVars(n, n, vtype=GRB.CONTINUOUS, name="w") # amount produced in period i to satisfy demand in period t
    
    c = np.zeros(n) # modified costs
    for t in range(n):
        c[t] = p[t] + sum(h[i] for i in range(t, n))

    # Set objective function  
    K = gp.quicksum(h[t] * gp.quicksum(d[i] for i in range(t + 1)) for t in range(n))
    obj_fn = gp.quicksum(f[t] * x[t] for t in range(n)) + \
             gp.quicksum(gp.quicksum(c[i] * w[i, t] for t in range(i, n)) for i in range(n)) \
             - K
    ULS_model.setObjective(obj_fn, GRB.MINIMIZE)

    # Add constraints
    for t in range(n):
        ULS_model.addConstr(gp.quicksum(w[i, t] for i in range(t + 1)) == d[t], name="demand")
        for i in range(n):
            if (i <= t):
                ULS_model.addConstr(w[i, t] >= 0, name="w_lb")
                ULS_model.addConstr(w[i, t] <= d[t] * x[i], name="w_ub")
    
    # LP relaxation case
    if (lp_relax == 1):
        # ULS_model.relax()          # Creates relaxation of MIP model by transforming integer var into continuous var
        ULS_model.Params.Method = 1  # Use dual simplex for LP relaxation
#        for i in range(n):          # manual relaxation
#            ULS_model.addConstr(x[i] >= 0, name="x_lb")
#            ULS_model.addConstr(x[i] <= 1, name="x_ub")

    # Solve model
    ULS_model.optimize()
                                    
    # Output results
    obj_opt = ULS_model.objVal
    
    print("\nOutput")
    print(60*'=')
                                    
    if ULS_model.status == GRB.OPTIMAL:
        print("Optimal objective value:", ULS_model.objVal)
        print("Production decisions (x):", [x[t].x for t in range(n)])
        print("Production for demand (w):", [[w[i, t].x for t in range(n)] for i in range(n)])
    else:
        print("No optimal solution found")
    print("\n")  
    
    return obj_opt

## Application

In [12]:
# Parameters
n = 5                          # number of time periods
p = [10, 15, 12, 8, 10]        # Variable cost for each period
h = [1, 1, 2, 1, 1]            # Storage cost for each period
f = [100, 100, 100, 100, 100]  # Fixed cost for each period
d = [20, 30, 25, 35, 30]       # Demand for each period

#### ULS

In [13]:
# ULS: Exact MIP
solve_ULS_1(f, p, h, d, n, M=200, lp_relax=0)


Output
Optimal objective value: 1580.0
Production decisions (x): [1.0, -0.0, -0.0, 1.0, -0.0]
Produced quantity (y): [75.0, 0.0, 0.0, 65.0, 0.0]
End-of-period inventory (s): [0.0, 55.0, 25.0, 0.0, 30.0, 0.0]




1580.0

In [6]:
# ULS: LP relaxation
solve_ULS_1(f, p, h, d, n, M=200, lp_relax=1)


Output
Optimal objective value: 1450.0
Production decisions (x): [0.25, 0.0, 0.125, 0.325, 0.0]
Produced quantity (y): [50.0, 0.0, 25.0, 65.0, 0.0]
End-of-period inventory (s): [0.0, 30.0, 0.0, 0.0, 30.0, 0.0]




1450.0

#### ULS extended

In [7]:
# Extended ULS: Exact MIP
solve_ULS_2(f, p, h, d, n, lp_relax=0)


Output
Optimal objective value: 1580.0
Production decisions (x): [1.0, 0.0, 0.0, 1.0, 0.0]
Production for demand (w): [[20.0, 30.0, 25.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 35.0, 30.0], [0.0, 0.0, 0.0, 0.0, 0.0]]




1580.0

In [8]:
# Extended ULS: LP relaxation
solve_ULS_2(f, p, h, d, n, lp_relax=1)


Output
Optimal objective value: 1580.0
Production decisions (x): [1.0, 0.0, 0.0, 1.0, 0.0]
Production for demand (w): [[20.0, 30.0, 25.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 35.0, 30.0], [0.0, 0.0, 0.0, 0.0, 0.0]]




1580.0

### Comparison

In [9]:
# Parameters generation
n = 10 # number of periods
num_exp = 100 # 100 experiments

average_quality_1 = 0
average_quality_2 = 0

for k in range(num_exp):
    f = np.zeros(n) # fixed costs of producing in period t
    p = np.zeros(n) # unit production cost in period t
    h = np.zeros(n) # unit storage cost in period t
    d = np.zeros(n) # demands in period t

    for t in range(n):
        f[t] = random.randint(1,100)
        p[t] = random.randint(1,100)
        h[t] = random.randint(1,100)
        d[t] = random.randint(1,100)

    M = 1000
    
    print('ULS 1')
    obj_opt_1 = solve_ULS_1(f, p, h, d, n, M, 0)
    obj_opt_relaxed_1 = solve_ULS_1(f, p, h, d, n, M, 1)
    average_quality_1 += obj_opt_1 / obj_opt_relaxed_1 /num_exp
    
    print('ULS 2 (extended)')
    obj_opt_2 = solve_ULS_2(f, p, h, d, n, 0)
    obj_opt_relaxed_2 = solve_ULS_2(f, p, h, d, n, 1)
    average_quality_2 += obj_opt_2 / obj_opt_relaxed_2 /num_exp

ULS 1

Output
Optimal objective value: 12699.0
Production decisions (x): [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
Produced quantity (y): [56.0, 104.0, 0.0, 56.0, 56.0, 91.0, 18.0, 56.0, 74.0, 0.0]
End-of-period inventory (s): [0.0, 0.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 46.0, 0.0]



Output
Optimal objective value: 12173.177
Production decisions (x): [0.056, 0.104, 0.0, 0.056, 0.056, 0.091, 0.018, 0.056, 0.074, 0.0]
Produced quantity (y): [56.0, 104.0, 0.0, 56.0, 56.0, 91.0, 18.0, 56.0, 74.0, 0.0]
End-of-period inventory (s): [0.0, 0.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 46.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 12699.0
Production decisions (x): [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
Production for demand (w): [[56.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 56.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 56.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0,


Output
Optimal objective value: 32972.0
Production decisions (x): [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Produced quantity (y): [72.0, 116.0, 0.0, 0.0, 84.0, 20.0, 70.0, 80.0, 22.0, 51.0]
End-of-period inventory (s): [0.0, 0.0, 54.0, 35.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]



Output
Optimal objective value: 32525.283
Production decisions (x): [0.072, 0.116, 0.0, 0.0, 0.084, 0.02, 0.07, 0.08, 0.022, 0.051]
Produced quantity (y): [72.0, 116.0, 0.0, 0.0, 84.0, 20.0, 70.0, 80.0, 22.0, 51.0]
End-of-period inventory (s): [0.0, 0.0, 54.0, 35.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 32972.0
Production decisions (x): [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Production for demand (w): [[72.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 62.0, 19.0, 35.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 84.0, 0.


Output
Optimal objective value: 36763.0
Production decisions (x): [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Produced quantity (y): [74.0, 185.0, 0.0, 42.0, 94.0, 56.0, 185.0, 0.0, 21.0, 3.0]
End-of-period inventory (s): [0.0, 0.0, 86.0, 0.0, 0.0, 0.0, 0.0, 89.0, 0.0, 0.0, 0.0]



Output
Optimal objective value: 36351.044
Production decisions (x): [0.074, 0.185, 0.0, 0.042, 0.094, 0.056, 0.185, 0.0, 0.021, 0.003]
Produced quantity (y): [74.0, 185.0, 0.0, 42.0, 94.0, 56.0, 185.0, 0.0, 21.0, 3.0]
End-of-period inventory (s): [0.0, 0.0, 86.0, 0.0, 0.0, 0.0, 0.0, 89.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 36763.0
Production decisions (x): [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Production for demand (w): [[74.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 99.0, 86.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 94.0,


Output
Optimal objective value: 40740.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Production for demand (w): [[52.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 89.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 97.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 11.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 72.0, 92.0, 65.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 89.0]]



Output
Optimal objective value: 40740.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Production for demand (w): [[52.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 89.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 97.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 


Output
Optimal objective value: 21091.191
Production decisions (x): [0.018, 0.048, 0.147, 0.0, 0.052, 0.094, 0.04, 0.067, 0.022, 0.09]
Produced quantity (y): [18.0, 48.0, 147.0, 0.0, 52.0, 94.0, 40.0, 67.0, 22.0, 90.0]
End-of-period inventory (s): [0.0, 0.0, 0.0, 69.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 21567.0
Production decisions (x): [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Production for demand (w): [[18.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 78.0, 69.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 52.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 94.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 40.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 67.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 90.0]

...


Output
Optimal objective value: 24186.787
Production decisions (x): [0.078, 0.071, 0.049, 0.029, 0.06, 0.08, 0.041, 0.013, 0.047, 0.096]
Produced quantity (y): [78.0, 71.0, 49.0, 29.0, 60.0, 80.0, 41.0, 13.0, 47.0, 96.0]
End-of-period inventory (s): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 24788.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Production for demand (w): [[78.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 71.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 49.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 29.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 60.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 80.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 41.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 96.0


Output
Optimal objective value: 17447.0
Production decisions (x): [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
Produced quantity (y): [27.0, 116.0, 0.0, 0.0, 78.0, 0.0, 30.0, 15.0, 58.0, 95.0]
End-of-period inventory (s): [0.0, 0.0, 84.0, 26.0, 0.0, 46.0, 0.0, 0.0, 0.0, 0.0, 0.0]



Output
Optimal objective value: 17047.72
Production decisions (x): [0.027, 0.116, 0.0, 0.0, 0.078, 0.0, 0.03, 0.015, 0.058, 0.095]
Produced quantity (y): [27.0, 116.0, 0.0, 0.0, 78.0, 0.0, 30.0, 15.0, 58.0, 95.0]
End-of-period inventory (s): [0.0, 0.0, 84.0, 26.0, 0.0, 46.0, 0.0, 0.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 17447.0
Production decisions (x): [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
Production for demand (w): [[27.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 32.0, 58.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 32.0, 46.


Output
Optimal objective value: 32330.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]
Produced quantity (y): [93.0, 63.0, 56.0, 60.0, 84.0, 112.0, 0.0, 4.0, 27.0, 53.0]
End-of-period inventory (s): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.0, 0.0, 0.0, 0.0, 0.0]



Output
Optimal objective value: 32029.772
Production decisions (x): [0.093, 0.063, 0.056, 0.06, 0.084, 0.112, 0.0, 0.004, 0.027, 0.053]
Produced quantity (y): [93.0, 63.0, 56.0, 60.0, 84.0, 112.0, 0.0, 4.0, 27.0, 53.0]
End-of-period inventory (s): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.0, 0.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 32330.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]
Production for demand (w): [[93.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 63.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 56.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 60.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 84.0, 


Output
Optimal objective value: 25453.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Produced quantity (y): [21.0, 73.0, 92.0, 73.0, 25.0, 52.0, 113.0, 0.0, 31.0, 56.0]
End-of-period inventory (s): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 76.0, 0.0, 0.0, 0.0]



Output
Optimal objective value: 24986.715
Production decisions (x): [0.021, 0.073, 0.092, 0.073, 0.025, 0.052, 0.113, 0.0, 0.031, 0.056]
Produced quantity (y): [21.0, 73.0, 92.0, 73.0, 25.0, 52.0, 113.0, 0.0, 31.0, 56.0]
End-of-period inventory (s): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 76.0, 0.0, 0.0, 0.0]


ULS 2 (extended)

Output
Optimal objective value: 25453.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Production for demand (w): [[21.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 73.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 92.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 73.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 25.


Output
Optimal objective value: 17765.0
Production decisions (x): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Production for demand (w): [[48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 88.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 53.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 49.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 28.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 28.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 67.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0]]


ULS 1

Output
Optimal objective value: 23228.0
Production decisions (x): [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]
Produced quantity (y): [47.0, 190.0, 0.0, 87.0, 95.0, 136.0, 0.0, 46.0, 45.0, 36.0]
End-of-period inventory (s): [0.0, 0.0, 95.0, 0.0, 0.0, 0.0, 72.0, 0.0, 0.0, 0.0, 0.0]



Output
Optimal objective 

In [10]:
print('Average Quality Ratios:')
print('average_quality_1: ', average_quality_1)
print('average_quality_2: ', average_quality_2)

Average Quality Ratios:
average_quality_1:  1.0185951956974553
average_quality_2:  1.0000000000000007


As we can see, average_quality_2 (extended ULS model 2) is much closer to 1 than average_quality_1 (basic ULS model 1). \
Hence, the set of feasible solutions for the extended ULS model 2 ($P_2$) is smaller than the set of ULS model 1 ($P_1$) which leads to a more efficient optimization with the extended ULS model 2. \
Since average_quality_2 is extremely close to 1, it supports the statement on p.18 of the textbook, saying that $P_2$ describes the convex hull of solutions to the problem. Meaning, that the extended problem formulation is basically the same as the LP relaxation.

Therefore, as described on p.18 of the textbook, $P_2$ $\subset$ $P_1$ and consequently $P_2$ is a better formulation than $P_1$ and it is also an ideal formulation.